# 0

In [1]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
import re
from datetime import datetime

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

# extracción de urls

In [2]:
urls = []
flag = 0
flag2 = 0

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# para cada una de las categorías principales
# poner un max excesivo
for category in range(1,31):

    while 1:

        # intenta clickar
        try:
            
            driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/a').click()
            flag2 = 0

            for subcategory in range (1,31):

                while 1:

                    try:
                        
                        urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/ul/div[{subcategory}]/a').get_attribute("href"))
                        sleep(0.1)
                        break

                    except:

                        flag2 = 1
                        break
                    
                if flag2:
                    break
            
            sleep(1)
            break
        
        # break si se acaban
        except:

            flag = 1
            break

    if flag:

        break

driver.quit()

# scrapeo de productos

In [ ]:
resultados_dia = {'supermarket': [],
                'category': [],
                'subcategory': [],
                'name': [], 
                'price': [], 
                'reference_price': [],
                'reference_unit': [],
                'insert_date': []}

total = 0
botones = []

# creación del driver de chrome
driver = webdriver.Chrome()

# para cada uno de los urls que han salido del primer Selenium
for u in urls:

        # get
        driver.get(u)
        sleep(3)

        # acepta las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(1)

        # parsea html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # listas resultados primera página
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})

        # cálculo del nº total de productos existentes en el super
        try:
                num = soup.find('span', {"class": "plp-breadcrumb__total-items"}).text

        except:
                num = 0

        total += int(re.findall('\d+', str(num))[0])
        
        # apendea los resultados de las listas al dictio extrayendo texto de labels y retocando con métodos de strings
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€').replace(',', '.'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' (').replace(',', '.'))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        # intenta clickar el botón
                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(1)
                        
                        except:
                                # para cuando hay botones flecha (+5 páginas)
                                try:
                                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/a[2]').click()
                                        sleep(1)
                                except:
                                        pass

                                pass
                        
                        # SE REPITE EL PROCESO ANTERIOR PARA EL RESTO DE PÁGINAS:
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(1)

                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))

                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€').replace(',', '.'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' (').replace(',', '.'))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        

        # resetea nº botones
        botones = []

# cierra driver
driver.quit()

# to csv
df = pd.DataFrame(resultados_dia)
fecha = datetime.today().strftime('%Y-%m-%d %H-%M-%S')
df.to_csv(f"../scrapeo({fecha}).csv", index=True, sep= ",")

# check total
print(f'Productos scrapeados: {len(resultados_dia["name"])} de {total}')